# Anomalies

In [1]:
from collections import OrderedDict

import jinja2
import os
import pandas as pd
# from scipy.optimize import minimize
import statsmodels.api as sm

#%matplotlib inline

C:\Users\samth\Miniconda3\envs\econ36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
factors = pd.read_csv("C:/Data/Thesis/_Factors.csv")

myfactors = pd.read_csv("C:/Data/Thesis/_MyFactors.csv")
myfactors.iloc[:,1:] = myfactors.iloc[:,1:]*100

f = pd.concat([factors, myfactors[["HML", "HMLm", "PMU06", "PMU16", "PMU", "CMA", "WML"]]], axis=1)
f.date = pd.to_datetime(f.date.astype("str"), format="%Y%m")
f = f.set_index("date")

f.head()

,rf,rm,smb,hml,rmw,cma,wml,hmlm,HML,HMLm,PMU06,PMU16,PMU,CMA,WML
date,,,,,,,,,,,,,,,
1963-07-01,0.27,-0.39,-0.46,-0.81,0.72,-1.16,0.99,-0.59,-0.721155,-0.864465,0.888426,0.739828,0.355650,-1.132040,0.550726
1963-08-01,0.25,5.07,-0.81,1.65,0.42,-0.40,1.08,0.28,1.000628,-0.060107,0.212301,0.470122,0.456718,-0.070141,1.443902
1963-09-01,0.27,-1.57,-0.48,0.19,-0.80,0.23,0.13,0.27,1.045719,1.463823,-1.169019,-1.130309,-0.665664,-0.361107,0.267052
1963-10-01,0.29,2.53,-1.29,-0.09,2.75,-2.26,3.14,-2.11,-0.549914,-1.635921,2.612164,3.006009,3.007174,-1.886318,3.648531
1963-11-01,0.27,-0.85,-0.85,1.71,-0.34,2.22,-0.75,0.91,1.288533,1.400487,-0.099125,-0.481188,-0.625498,0.314912,-1.489579


In [3]:
models = {
    "CAPM": {"f": ["rm",],
             "regstr": r'R^i=a^i+\beta^iMkt',
             "coefs": ["a", "b"],
             "display_coefs": ["a", "b"]},
    "F93": {"f": ["rm", "smb", "hml"],
               "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal',
               "coefs": ["a", "b", "s", "v"],
               "display_coefs": ["a", "b", "s", "v"]},
    "C97": {"f" : ["rm", "smb", "hml", "wml"],
              "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom',
              "coefs": ["a", "b", "s", "v", "m"],
              "display_coefs": ["a", "v", "m"]},
    "C97b": {"f" : ["rm", "smb", "HMLm", "wml"],
               "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom',
               "coefs": ["a", "b", "s", "v", "m"],
               "display_coefs": ["a", "v", "m"]},
    "F15": {"f" : ["rm", "smb", "hml", "rmw", "cma"],
               "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom',
               "coefs": ["a", "b", "s", "v", "p", "i"],
              "display_coefs": ["a", "v", "p", "i"]},
    "F16": {"f" : ["rm", "smb", "hml", "PMU", "cma"],
               "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
               "coefs": ["a", "b", "s", "v", "p", "i"],
              "display_coefs": ["a", "v", "p", "i"]},
    "F17": {"f" : ["rm", "smb", "hml", "PMU", "cma", "wml"],
               "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv+m^iMom',
               "coefs": ["a", "b", "s", "v", "p", "i", "m"],
              "display_coefs": ["a", "v", "p", "i", "m"]},
    "B16": {"f" : ["rm", "smb", "HMLm", "wml", "PMU"],
              "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
              "coefs": ["a", "b", "s", "v", "m", "p"],
              "display_coefs": ["a", "v", "m", "p"]},
    "B16b": {"f" : ["rm", "smb", "HMLm", "wml", "PMU", "cma"],
              "regstr": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf+i^iInv',
              "coefs": ["a", "b", "s", "v", "m", "p", "i"],
              "display_coefs": ["a", "v", "m", "p", "i"]},
}

In [18]:
def gen_K_I_J_tbl_dict(LHS, f, factors_list, start="1963-07", end="2017-12",
                 coef_display_names=None, K=1, I=5, J=5, transpose_tuple=None):

    # RHS (independent variables i.e. the model) is the same for each anomaly
    # sm.add_constant puts a column named "const" as the first column in the DataFrame
    rhs = sm.add_constant(f[start:end][factors_list])

    tbl_dict = OrderedDict([
        (coef, {"display": coef, "coef": [], "tval": []}) for coef in rhs.columns
    ])

    if coef_display_names:
        if len(coef_display_names)==(len(factors_list)+1):
            for coefi, coef in enumerate(rhs.columns):
                tbl_dict[coef]["display"] = coef_display_names[coefi]
        else:
            print("tbl_dictlert: len(coef_display_names)!=(len(factors_list)+1)",
                  "Don't forget the intercept also needs a display name",
                  "Using factors_list as coef_display_names")

    # grab the coefficients and t-values for each anomaly portfolio
    # in the end, we'll have a list of len(LHS) coefs and tvals
    # for each anomaly portfolio
    for lhs in LHS.columns:

        lhs = LHS[start:end][lhs].fillna(-99.99) - f[start:end]["rf"]       # DataFrame

        # regress anomaly on factors+constant
        model = sm.OLS(lhs, rhs)
        fit = model.fit()

        # get the coefficients and their t-values for 1 anomaly
        # this is in the order of factors_list
        coef = ["{:.2f}".format(b) for b in fit.params]
        tval = ["{:.2f}".format(t) for t in fit.tvalues]

        for factori, factor in enumerate(rhs.columns):
            tbl_dict[factor]["coef"].append(coef[factori])
            tbl_dict[factor]["tval"].append(tval[factori])

    # for each coefficient turn the list of coefs and tvals into a KxIxJ array
    for coef in tbl_dict:
        tbl_dict[coef]["coef"] = pd.np.array(tbl_dict[coef]["coef"]).reshape(K, I, J)
        tbl_dict[coef]["tval"] = pd.np.array(tbl_dict[coef]["tval"]).reshape(K, I, J)

        if transpose_tuple:
            tbl_dict[coef]["coef"] = tbl_dict[coef]["coef"].transpose(transpose_tuple)
            tbl_dict[coef]["tval"] = tbl_dict[coef]["tval"].transpose(transpose_tuple)

    return tbl_dict

In [5]:
path = "C:/Users/samth/Dropbox/Thesis/Tex/Tables/"

env = jinja2.Environment(
    block_start_string='-%', block_end_string='%-',
    variable_start_string='=%', variable_end_string='%=',
    loader=jinja2.FileSystemLoader(path)
)

In [6]:
n_months, start, end = 654, "1963-07", "2017-12"

In [7]:
template = env.get_template("_K_I_J_template.txt")

## BM Constrained

In [19]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"), 1, 5, 5
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "25_Size_BM", r"BM"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=None)  # ((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "4", "High"],
        "row_names": ["Small", "2", "3", "4", "Big"],
        # "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_25_Size_BM_196307_201712.tex
F16_25_Size_BM_196307_201712.tex
F17_25_Size_BM_196307_201712.tex


In [20]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_INV_2x4x4.CSV"), 2, 4, 4
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "32_Size_BM_Inv", r"BM"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "High"],
        "row_names": ["Low Inv", "2", "3", "High Inv"],
        "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_32_Size_BM_Inv_196307_201712.tex
F16_32_Size_BM_Inv_196307_201712.tex
F17_32_Size_BM_Inv_196307_201712.tex


In [21]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_OP_2x4x4.CSV"), 2, 4, 4
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "32_Size_BM_OP", r"BM"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "High"],
        "row_names": [r"Low $\text{OP}^{06}$", "2", "3", r"High $\text{OP}^{06}$"],
        "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_32_Size_BM_OP_196307_201712.tex
F16_32_Size_BM_OP_196307_201712.tex
F17_32_Size_BM_OP_196307_201712.tex


In [22]:
a, K, I, J = pd.read_csv("C:/Data/Thesis/32_Portfolios_BMm_Prior.csv"), 2, 4, 4
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "32_Size_BMm_Prior", r"$\text{BM}^{m}$"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "High"],
        "row_names": [r"Low Prior", "2", "3", r"High Prior"],
        "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_32_Size_BMm_Prior_196307_201712.tex
F16_32_Size_BMm_Prior_196307_201712.tex
F17_32_Size_BMm_Prior_196307_201712.tex


## Other Anomalies

In [27]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_INV_5x5.CSV"), 1, 5, 5
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "25_Size_Inv", r"Inv"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=None)  # ((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "4", "High"],
        "row_names": [r"Small", "2", "3", "4", r"Big"],
        # "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_25_Size_Inv_196307_201712.tex
F16_25_Size_Inv_196307_201712.tex
F17_25_Size_Inv_196307_201712.tex


In [25]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_OP_INV_2x4x4.CSV"), 2, 4, 4
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "32_Size_OP_Inv", r"$\text{OP}^{06}$"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "High"],
        "row_names": [r"Low Inv", "2", "3", r"High Inv"],
        "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_32_Size_OP_Inv_196307_201712.tex
F16_32_Size_OP_Inv_196307_201712.tex
F17_32_Size_OP_Inv_196307_201712.tex


In [28]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_AC_5x5.csv"), 1, 5, 5
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "25_Size_Acc", r"Acc"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=None)  # ((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "4", "High"],
        "row_names": [r"Small", "2", "3", "4", r"Big"],
        # "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_25_Size_Acc_196307_201712.tex
F16_25_Size_Acc_196307_201712.tex
F17_25_Size_Acc_196307_201712.tex


In [29]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_BETA_5x5.csv"), 1, 5, 5
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "25_Size_Beta", r"Beta"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=None)  # ((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "4", "High"],
        "row_names": [r"Small", "2", "3", "4", r"Big"],
        # "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_25_Size_Beta_196307_201712.tex
F16_25_Size_Beta_196307_201712.tex
F17_25_Size_Beta_196307_201712.tex


In [30]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_NI_5x5.csv"), 1, 5, 7
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "25_Size_NI", r"NI"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=None)  # ((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Neg", "0", "Low", "2", "3", "4", "High"],
        "row_names": [r"Small", "2", "3", "4", r"Big"],
        # "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\scriptsize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_25_Size_NI_196307_201712.tex
F16_25_Size_NI_196307_201712.tex
F17_25_Size_NI_196307_201712.tex


In [31]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_Prior_12_2.CSV"), 1, 5, 5
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "25_Size_Prior", r"Prior"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=None)  # ((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "4", "High"],
        "row_names": [r"Small", "2", "3", "4", r"Big"],
        # "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_25_Size_Prior_196307_201712.tex
F16_25_Size_Prior_196307_201712.tex
F17_25_Size_Prior_196307_201712.tex


In [32]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_RESVAR_5x5.csv"), 1, 5, 5
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "25_Size_RVar", r"RVar"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=None)  # ((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "4", "High"],
        "row_names": [r"Small", "2", "3", "4", r"Big"],
        # "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_25_Size_RVar_196307_201712.tex
F16_25_Size_RVar_196307_201712.tex
F17_25_Size_RVar_196307_201712.tex


In [33]:
a, K, I, J = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_VAR_5x5.csv"), 1, 5, 5
a = a.iloc[:n_months, 1:]
a.index = f.index

fname_suffix, col_name = "25_Size_Var", r"Var"

for m in ["B16", "F16", "F17"]:
    tbl_dict = gen_K_I_J_tbl_dict(a, f, models[m]["f"], start=start, end=end,
                                  coef_display_names=models[m]["coefs"],
                                  K=K, I=I, J=J, transpose_tuple=None)  # ((0, 2, 1)))

    context = {
        "coefs": tbl_dict, "K":K, "I": I, "J": J,
        "col_name": col_name,
        "col_names": ["Low", "2", "3", "4", "High"],
        "row_names": [r"Small", "2", "3", "4", r"Big"],
        # "K_hdrs": ["Small", "Big"],
        "regstr": models[m]["regstr"],
        "caption": "{} Portfolios with {} Factors {} through {}".format(fname_suffix.replace("_", "-"), m, start, end),
        "label": "tbl:{}_{}".format(fname_suffix, m),
        "font_size": r"\footnotesize",
    }

    fname = "{}_{}_{}_{}.tex".format(m, fname_suffix, start.replace("-", ""), end.replace("-", ""))

    with open(os.path.join(path, fname), "w") as table:
        table.write(template.render(context))
    print(fname)

B16_25_Size_Var_196307_201712.tex
F16_25_Size_Var_196307_201712.tex
F17_25_Size_Var_196307_201712.tex


# Annual value with monthly value

In [ ]:
a = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV")

In [ ]:
a.date = pd.to_datetime(a.date.astype(str), format="%Y%m")
a = a.set_index("date")

In [ ]:
fit = sm.OLS(a["SMALL LoBM"], sm.add_constant(f[["rm", "smb", "hmlm", "wml", "PMU"]])).fit()
pd.concat([fit.params, fit.pvalues], axis=1).round(2)

In [ ]:
fit = sm.OLS(a["SMALL LoBM"], sm.add_constant(f[["rm", "smb", "hmlm", "hml", "wml", "PMU"]])).fit()
pd.concat([fit.params, fit.pvalues], axis=1).round(2)

In [ ]:
fit = sm.OLS(a["SMALL HiBM"], sm.add_constant(f[["rm", "smb", "hmlm", "wml", "PMU"]])).fit()
pd.concat([fit.params, fit.pvalues], axis=1).round(2)

In [ ]:
fit = sm.OLS(a["SMALL HiBM"], sm.add_constant(f[["rm", "smb", "hmlm", "hml", "wml", "PMU"]])).fit()
pd.concat([fit.params, fit.pvalues], axis=1).round(2)